# ETF - Strategy Backtesting

### Strategy:
1. At the start of each week, check the 30 week (6 months) rate of change of NiftyBees, GoldBees, BankBees and ONGC.
2. If none of the ETFs are returning more than 3% in 6 months. Do nothing. Be invested in LiquidBees.
3. If one or both ETFs are returning more than 3% invest your entire position in that ETF. (Higher Performance ETF).

In [ ]:
import pandas as pd
import quandl

In [ ]:
quandl_key = open("data/quandlkey").readline().rstrip()
quandl.ApiConfig.api_key = quandl_key

In [ ]:
df = quandl.get(["NSE/NIFTYBEES.5",
                 "NSE/GOLDBEES.5",
                 "NSE/LIQUIDBEES.5",
                 "NSE/BANKBEES.5",
                 "NSE/ONGC.5"]).rename(columns={'NSE/NIFTYBEES - Close': 'nc',
                                                'NSE/GOLDBEES - Close': 'gc',
                                                'NSE/LIQUIDBEES - Close': 'lc',
                                                'NSE/BANKBEES - Close': 'bc',
                                                'NSE/ONGC - Close': 'oc'})

In [ ]:
df = df.dropna()
df['wn'] = df.index.weekofyear
print(df)

### Data Wrangling - Daily to Weekly

Data wrangling performed to compress the large dataset to retain only the essential data points (week beginning and ending).
* week ending - for calculating Rate of Change (RoC)
* week beginning - for trade actions (buy/sell)

In [ ]:
# Converting data from df to weekly format

weekly_col = ['wb','we','nb','ne','gb','ge','lb','le', 'bb', 'be', 'ob', 'oe']
weekly = pd.DataFrame(columns=weekly_col)
curr_week = -1
df_prev_week = 0
# cw_data - current week data
cw_data = dict.fromkeys(weekly_col,0.0)

for row in range(len(df)):
    # cw_ds - current week dataset
    cw_ds = df.iloc[row]
    if df_prev_week == cw_ds[5]:
        cw_data['we'] = df.index[row]
        cw_data['ne'] = cw_ds[0]
        cw_data['ge'] = cw_ds[1]
        cw_data['le'] = cw_ds[2]
        cw_data['be'] = cw_ds[3]
        cw_data['oe'] = cw_ds[4]
    else:
        weekly.loc[curr_week] = cw_data
        curr_week += 1
        cw_data['wb'] = cw_data['we'] = df.index[row]
        cw_data['nb'] = cw_data['ne'] = cw_ds[0]
        cw_data['gb'] = cw_data['ge'] = cw_ds[1]
        cw_data['lb'] = cw_data['le'] = cw_ds[2]
        cw_data['bb'] = cw_data['be'] = cw_ds[3]
        cw_data['ob'] = cw_data['oe'] = cw_ds[4]
    df_prev_week = cw_ds[5]

weekly.loc[curr_week] = cw_data
weekly.drop([-1], inplace=True)
print("Weekly Data:")
weekly['wb'] = pd.to_datetime(weekly['wb'])
weekly['we'] = pd.to_datetime(weekly['we'])
print(weekly)

### Trade Simulation

In [ ]:
initial_funds = 10000.00
fund_bal = initial_funds
trade_book = pd.DataFrame(columns = ['Date','Scrip','Value','Quantity','Amount'])
# trade book entry
tb_entry = dict.fromkeys(['Date','Scrip','Value','Quantity','Amount'])
period = 30
nbr = gbr = bbr = onr = 0
entry_num = 0

def roc_calc(col):
    initial = weekly.loc[week_num - period][col]
    final = weekly.loc[week_num - 1][col]
    return (final - initial) * 100 / initial

def invest_in(scrip):
    
    def entry_scrip():
        global entry_num, fund_bal
        tb_entry['Date'] = weekly.wb.values[week_num]
        tb_entry['Scrip'] = scrip
        tb_entry['Value'] = weekly[scrip+'b'].values[week_num]
        tb_entry['Quantity'] = int(fund_bal / tb_entry['Value'])
        tb_entry['Amount'] = tb_entry['Value'] * tb_entry['Quantity'] * -1
        fund_bal += tb_entry['Amount']
        # print(tb_entry, fund_bal)
        trade_book.loc[entry_num] = tb_entry
        entry_num += 1
    def exit_scrip():
        global entry_num, fund_bal
        tb_entry['Date'] = weekly.wb.values[week_num]
        tb_entry['Scrip'] = trade_book.Scrip.values[-1]
        tb_entry['Value'] = weekly[trade_book.Scrip.values[-1]+'b'].values[week_num]
        tb_entry['Quantity'] = trade_book.Quantity.values[-1]
        tb_entry['Amount'] = tb_entry['Value'] * tb_entry['Quantity']
        fund_bal += tb_entry['Amount']
        # print(tb_entry, fund_bal)
        trade_book.loc[entry_num] = tb_entry
        entry_num += 1
    
    if entry_num == 0:
        entry_scrip()
    elif scrip != trade_book.Scrip.values[-1]:
        exit_scrip()
        entry_scrip()
        

for week_num in range(period,len(weekly)):
    nbr = roc_calc('ne')
    gbr = roc_calc('ge')
    bbr = roc_calc('be')
    onr = roc_calc('oe')
    
    if nbr > 3:
        if nbr > gbr and nbr > bbr and nbr > onr:
            invest_in('n')
    elif gbr > 3:
        if gbr > bbr and gbr > onr:
            invest_in('g')
    elif bbr > 3:
        if bbr > onr:
            invest_in('b')
    elif onr > 3:
        invest_in('o')
    else:
        invest_in('l')
        
print(trade_book)

### Trade Analysis

In [ ]:
ltp = weekly[trade_book.Scrip.values[-1]+'e'].values[-1]
namt = ltp * trade_book.Quantity.values[-1]
nbal = namt + fund_bal

odd_dates = pd.Series(trade_book.Date.iloc[1::2].values)
odd_dates.loc[len(odd_dates)] = weekly.we.values[-1]
even_dates = pd.Series(trade_book.Date.iloc[::2].values)
odd_amt = pd.Series(trade_book.Amount.iloc[1::2].values)
odd_amt.loc[len(odd_amt)] = namt
even_amt = pd.Series(trade_book.Amount.iloc[::2].values)

trade_stats = pd.DataFrame()
trade_stats['EntryDate'] = even_dates
trade_stats['ExitDate'] = odd_dates
trade_stats['HoldPeriod'] = odd_dates - even_dates
trade_stats['Scrip'] = pd.Series(trade_book.Scrip.iloc[::2].values)
trade_stats['EntryAmt'] = even_amt
trade_stats['ExitAmt'] = odd_amt
trade_stats['Gains'] = odd_amt + even_amt
print(trade_stats)

In [ ]:
np = trade_stats.Gains.sum()
cagr = (((nbal / initial_funds) ** (1 / 10) -1) * 100).round(2)

print("Initial Funds:\t{0:.2f}".format(initial_funds))
print("Net Profit:\t{0:.2f}".format(np))
print("CAGR:\t\t{0:8.3}%".format(cagr))